In [68]:
!pip -q install -U scikit-learn pandas numpy openpyxl xarray netCDF4 matplotlib plotly tensorflow


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from calendar import monthrange
import xarray as xr
import pandas as pd
import plotly.express as px
import plotly.io as pio
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import plotly.graph_objects as go
from typing import Literal, Optional, Dict, Any
from sklearn.preprocessing import StandardScaler, OneHotEncoder
pio.renderers.default = "notebook_connected"  # ou "vscode", "jupyterlab", etc.

## Extração dos Dados

### Download dos dados a partir do earthdata

Para utilizar devidamente esse script é necessário se cadastrar em https://urs.earthdata.nasa.gov, após isso, crie um .env com suas credenciais para fazer o download desse dataset, ajuste as variáveis iniciais de acordo com o folder desejado e range da data de download. Foi implementado paralelismo para acelerar o processo de download, ajuste conforme necessário.

In [65]:
import os
import sys
import time
import netrc
import pathlib
import platform
from datetime import datetime, timedelta
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Iterable, Tuple, List, Optional

import requests


START = "2015-01-01"     # YYYY-MM-DD
END   = "2017-12-31"     # YYYY-MM-DD (inclusivo)
OUTDIR = "./dados_gpcp"  # pasta de saída
WORKERS = 4              # paralelismo
SKIP_EXISTING = True     # pular arquivos já existentes

# -------------------- Constantes do produto --------------------
EARTHDATA_HOST = "urs.earthdata.nasa.gov"
PRODUCT_ROOT = "https://data.gesdisc.earthdata.nasa.gov/data/GPCP/GPCPDAY.3.3"
VERSION_TAG = "V3.3"  # versão do produto

# --- Loader .env (zero-deps) ---
def load_dotenv_simple(path: str = ".env") -> None:
    p = pathlib.Path(path)
    if not p.exists():
        print(f"[aviso] .env não encontrado em {p.resolve()}")
        return
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        os.environ.setdefault(k.strip(), v.strip().strip('"').strip("'"))

load_dotenv_simple(".env")

# -------------------- Helpers --------------------
def get_credentials() -> Tuple[Optional[str], Optional[str]]:
    """Tenta obter (user, pwd) de env vars -> NETRC explícito -> _netrc/.netrc."""
    user = os.getenv("EARTHDATA_USERNAME")
    pwd  = os.getenv("EARTHDATA_PASSWORD")
    if user and pwd:
        return user, pwd

    netrc_env = os.getenv("NETRC")
    if netrc_env and pathlib.Path(netrc_env).exists():
        try:
            a = netrc.netrc(netrc_env)
            c = a.authenticators(EARTHDATA_HOST)
            if c and len(c) >= 3:
                return c[0], c[2]
        except Exception:
            pass

    home = pathlib.Path.home()
    candidates = [home / ".netrc"]
    if platform.system().lower().startswith("win"):
        candidates.insert(0, home / "_netrc")

    for p in candidates:
        if p.exists():
            try:
                a = netrc.netrc(str(p))
                c = a.authenticators(EARTHDATA_HOST)
                if c and len(c) >= 3:
                    return c[0], c[2]
            except Exception:
                continue

    return None, None

def human(n: float) -> str:
    for unit in ['','K','M','G','T','P']:
        if abs(n) < 1024.0:
            return f"{n:3.1f}{unit}B"
        n /= 1024.0
    return f"{n:.1f}EB"

def filename_from_url(url: str) -> str:
    return pathlib.Path(urlparse(url).path).name or "download.bin"

def daterange(start: datetime, end: datetime) -> Iterable[datetime]:
    cur = start
    step = timedelta(days=1)
    while cur <= end:
        yield cur
        cur += step

def build_gpcp_url(day: datetime) -> str:
    yyyy = day.strftime("%Y")
    ymd  = day.strftime("%Y%m%d")
    fname = f"GPCPDAY_L3_{ymd}_{VERSION_TAG}.nc4"
    return f"{PRODUCT_ROOT}/{yyyy}/{fname}"

def session_with_auth(user: Optional[str]=None, pwd: Optional[str]=None, token: Optional[str]=None) -> requests.Session:
    """Cria sessão; se houver token, usa Bearer; senão, usa Basic (user,pwd)."""
    s = requests.Session()
    s.headers.update({"User-Agent": "iDEV-downloader/1.3 (Notebook)"})
    if token:
        s.headers["Authorization"] = f"Bearer {token}"
    elif user and pwd:
        s.auth = (user, pwd)
    return s

def earthdata_fetch(session: requests.Session, url: str, auth_pair: Tuple[Optional[str], Optional[str]], token: Optional[str]=None, timeout=60) -> requests.Response:
    """
    Faz GET já com a auth disponível.
    Se 401 apontar para fluxo OAuth do URS, sugere consent/login no navegador.
    """
    user, pwd = auth_pair
    if token:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout)
    elif user and pwd:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout, auth=(user, pwd))
    else:
        r = session.get(url, stream=True, allow_redirects=True, timeout=timeout)

    if r.status_code == 401 and "urs.earthdata.nasa.gov/oauth/authorize" in r.url:
        raise RuntimeError(
            "HTTP 401 via URS/OAuth. Se estiver usando token, verifique validade/escopo. "
            "Caso seja o primeiro acesso, abra a URL do dataset no navegador, faça login e autorize o domínio "
            "(consent) para sua conta Earthdata. Depois rode novamente."
        )
    return r

def download_one(url: str, outdir: pathlib.Path, skip_existing: bool, retries=3, backoff=2.0) -> pathlib.Path:
    outdir.mkdir(parents=True, exist_ok=True)
    outpath = outdir / filename_from_url(url)

    if skip_existing and outpath.exists() and outpath.stat().st_size > 0:
        print(f"[skip] {outpath.name} ({human(outpath.stat().st_size)})")
        return outpath

    user, pwd = get_credentials()
    token = os.getenv("EARTHDATA_TOKEN")
    s = session_with_auth(user, pwd, token)

    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = earthdata_fetch(s, url, (user, pwd), token=token)
            if r.status_code == 404:
                raise FileNotFoundError(f"HTTP 404 (não encontrado): {url}")
            if r.status_code != 200:
                raise RuntimeError(f"HTTP {r.status_code} – URL final: {r.url}")

            total = int(r.headers.get("Content-Length", 0))
            downloaded = 0
            start = time.time()

            tmp_path = outpath.with_suffix(outpath.suffix + ".part")
            with open(tmp_path, "wb") as f:
                for chunk_bytes in r.iter_content(chunk_size=1024*1024):
                    if not chunk_bytes:
                        continue
                    f.write(chunk_bytes)
                    downloaded += len(chunk_bytes)
                    if total > 0:
                        pct = 100 * downloaded / total
                        sys.stdout.write(f"\rBaixando {outpath.name}: {human(downloaded)}/{human(total)} ({pct:5.1f}%)")
                    else:
                        sys.stdout.write(f"\rBaixando {outpath.name}: {human(downloaded)}")
                    sys.stdout.flush()

            tmp_path.replace(outpath)
            dur = time.time() - start
            sys.stdout.write(f"\n[ok] {outpath.name} em {dur:.1f}s ({human(downloaded)})\n")
            return outpath

        except Exception as e:
            last_err = e
            if attempt < retries:
                wait = backoff ** (attempt - 1)
                print(f"[tentativa {attempt}/{retries} falhou] {e} → retry em {wait:.1f}s")
                time.sleep(wait)
            else:
                print(f"[erro] {e}")

    raise RuntimeError(f"Falha definitiva ao baixar: {url} → {last_err}")

# -------------------- Execução --------------------
try:
    start_dt = datetime.strptime(START, "%Y-%m-%d")
    end_dt   = datetime.strptime(END,   "%Y-%m-%d")
except ValueError as e:
    raise SystemExit(f"[erro] Datas inválidas: {e}\nUse o formato YYYY-MM-DD.")

if end_dt < start_dt:
    raise SystemExit("[erro] END não pode ser anterior a START.")

urls = [build_gpcp_url(d) for d in daterange(start_dt, end_dt)]
print(f"Total de arquivos a baixar: {len(urls)}")

outdir_path = pathlib.Path(OUTDIR).resolve()
failed: List[str] = []

if WORKERS <= 1:
    for u in urls:
        try:
            download_one(u, outdir_path, skip_existing=SKIP_EXISTING)
        except Exception as e:
            failed.append(f"{u} → {e}")
else:
    with ThreadPoolExecutor(max_workers=WORKERS) as pool:
        futs = {pool.submit(download_one, u, outdir_path, SKIP_EXISTING): u for u in urls}
        for fut in as_completed(futs):
            try:
                fut.result()
            except Exception as e:
                failed.append(f"{futs[fut]} → {e}")

print("\nConcluído.")
if failed:
    print("\nFalhas:")
    for item in failed:
        print(" -", item)


Total de arquivos a baixar: 1096
[skip] GPCPDAY_L3_20150101_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150102_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150103_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150104_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150106_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150105_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150107_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150108_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150109_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150110_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150112_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150113_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150111_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150114_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150115_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150117_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150116_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150118_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150120_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150119_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150121_V3.3.nc4 (1.5MB)
[skip] GPCPDAY_L3_20150123_V3.3.nc4 (1.5MB)

### Leitura e concatenação de todos nc4 em um único dataframe

In [66]:
import re
import os
import xarray as xr
from pathlib import Path

# -------------------- Config --------------------
FOLDER = "./dados_gpcp"            # pasta com os .nc4 baixados
VARIABLE = None                    
OUTPUT_CSV = "gpcp_daily_concat.csv"
OUTPUT_PARQUET = "gpcp_daily_concat.parquet"

NORMALIZE_LON = True               # converte lon para [-180, 180]
SORT_OUTPUT = True                 # ordena por (date, lat, lon)

#  bounding box do Pará – ative se quiser filtrar já aqui
USE_PA_BBOX = False
PA_BBOX = dict(lon_min=-60.5, lon_max=-46.0, lat_min=-9.8, lat_max=2.5)

# -------------------- Funções utilitárias --------------------
DATE_RE = re.compile(r"(\d{8})")  # captura YYYYMMDD no nome do arquivo

def parse_date_from_name(path: Path) -> np.datetime64:
    m = DATE_RE.search(path.name)
    if not m:
        raise ValueError(f"Não consegui extrair YYYYMMDD do nome: {path.name}")
    ymd = m.group(1)
    return np.datetime64(f"{ymd[:4]}-{ymd[4:6]}-{ymd[6:8]}")

def load_one_dataset(p: Path) -> xr.Dataset:
    ds = xr.open_dataset(p)
    # garante dimensão temporal (muitos diários já têm 'time' com 1 passo)
    if "time" not in ds.coords and "time" not in ds.dims:
        t = parse_date_from_name(p)
        ds = ds.expand_dims(time=[t])
    # assegura que time seja datetime64 ns
    if np.issubdtype(ds["time"].dtype, np.datetime64) is False:
        ds["time"] = xr.decode_cf(ds).get("time", ds["time"])
    return ds

def pick_main_var(ds: xr.Dataset, prefer: str | None = None) -> str:
    if prefer and prefer in ds.data_vars:
        return prefer
    ignore = {"mask", "lsmask", "land_sea_mask", "lat_bnds", "lon_bnds"}
    for v in ds.data_vars:
        if v.lower() in ignore:
            continue
        if ds[v].ndim >= 2:
            return v
    # fallback: a primeira variável qualquer
    return list(ds.data_vars)[0]

# -------------------- Coleta e leitura --------------------
files = sorted(Path(FOLDER).glob("*.nc4"))
if not files:
    raise SystemExit(f"Nenhum .nc4 encontrado em {Path(FOLDER).resolve()}")

datasets = []
failed = []
for i, p in enumerate(files, 1):
    try:
        ds = load_one_dataset(p)
        datasets.append(ds)
        print(f"[{i}/{len(files)}] ok → {p.name}")
    except Exception as e:
        failed.append((p.name, str(e)))
        print(f"[{i}/{len(files)}] erro → {p.name}: {e}")

if not datasets:
    raise SystemExit("Nenhum dataset pôde ser aberto.")



[1/1096] ok → GPCPDAY_L3_20150101_V3.3.nc4
[2/1096] ok → GPCPDAY_L3_20150102_V3.3.nc4
[3/1096] ok → GPCPDAY_L3_20150103_V3.3.nc4
[4/1096] ok → GPCPDAY_L3_20150104_V3.3.nc4
[5/1096] ok → GPCPDAY_L3_20150105_V3.3.nc4
[6/1096] ok → GPCPDAY_L3_20150106_V3.3.nc4
[7/1096] ok → GPCPDAY_L3_20150107_V3.3.nc4
[8/1096] ok → GPCPDAY_L3_20150108_V3.3.nc4
[9/1096] ok → GPCPDAY_L3_20150109_V3.3.nc4
[10/1096] ok → GPCPDAY_L3_20150110_V3.3.nc4
[11/1096] ok → GPCPDAY_L3_20150111_V3.3.nc4
[12/1096] ok → GPCPDAY_L3_20150112_V3.3.nc4
[13/1096] ok → GPCPDAY_L3_20150113_V3.3.nc4
[14/1096] ok → GPCPDAY_L3_20150114_V3.3.nc4
[15/1096] ok → GPCPDAY_L3_20150115_V3.3.nc4
[16/1096] ok → GPCPDAY_L3_20150116_V3.3.nc4
[17/1096] ok → GPCPDAY_L3_20150117_V3.3.nc4
[18/1096] ok → GPCPDAY_L3_20150118_V3.3.nc4
[19/1096] ok → GPCPDAY_L3_20150119_V3.3.nc4
[20/1096] ok → GPCPDAY_L3_20150120_V3.3.nc4
[21/1096] ok → GPCPDAY_L3_20150121_V3.3.nc4
[22/1096] ok → GPCPDAY_L3_20150122_V3.3.nc4
[23/1096] ok → GPCPDAY_L3_20150123_V3.3.n

In [67]:
df_test = datasets[0].to_dataframe()
print(df_test)

                                precip  probability_liquid_phase  time_bnds
time       lat    lon     bnds                                             
2015-01-01  89.75 -179.75 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                  -179.25 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                  -178.75 0        0.0                       0.0 2015-01-01
...                                ...                       ...        ...
           -89.75  178.75 1        0.0                       0.0 2015-01-02
                   179.25 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02
                   179.75 0        0.0                       0.0 2015-01-01
                          1        0.0                       0.0 2015-01-02

[518400 row

In [68]:
# -------------------- Concatenação temporal --------------------
# concatena ao longo de 'time' (cada arquivo diário vira 1 passo temporal)
ds_all = xr.concat(datasets, dim="time", join="outer", combine_attrs="override")

# escolhe a variável principal
var_name = pick_main_var(ds_all, prefer=VARIABLE)
print(f"Variável selecionada: {var_name}")

# renomeia para 'precip' (padrão), mantendo apenas o necessário
ds_main = ds_all[[var_name]].rename({var_name: "precip"})


# 1) ative o recorte e use o bbox do Pará (IBGE, extremos oficiais)
USE_PA_BBOX = True
PA_BBOX = dict(
    lon_min=-58.89833,  # Oeste  (−58°53′54″)
    lon_max=-46.06083,  # Leste  (−46°03′39″)
    lat_min=-9.84111,   # Sul    (−09°50′28″)
    lat_max= 2.59111    # Norte  (+02°35′28″)
)

# 2) normaliza longitude p/ [-180, 180] (mantém seu trecho)
if NORMALIZE_LON and ds_main.coords.get("lon") is not None:
    lon = ds_main["lon"]
    if lon.max() > 180:
        ds_main = ds_main.assign_coords(lon=(((lon + 180) % 360) - 180))

# 3) recorte robusto ao sentido das coordenadas
if USE_PA_BBOX and {"lon", "lat"}.issubset(ds_main.coords):
    # lon pode estar crescente (−180→+180) – normalmente está
    lon_vals = ds_main["lon"].values
    lon_slice = slice(PA_BBOX["lon_min"], PA_BBOX["lon_max"]) \
        if lon_vals[0] < lon_vals[-1] else slice(PA_BBOX["lon_max"], PA_BBOX["lon_min"])

    # lat às vezes vem decrescente (+90→−90);
    lat_vals = ds_main["lat"].values
    lat_slice = slice(PA_BBOX["lat_min"], PA_BBOX["lat_max"]) \
        if lat_vals[0] < lat_vals[-1] else slice(PA_BBOX["lat_max"], PA_BBOX["lat_min"])

    ds_main = ds_main.sel(lon=lon_slice, lat=lat_slice)


Variável selecionada: precip


In [69]:

# -------------------- DataFrame final --------------------
# Garante nomes padrão de coord
rename_coords = {}
for cand in ["latitude", "Latitude", "LAT"]:
    if "lat" not in ds_main.coords and cand in ds_main.coords:
        rename_coords[cand] = "lat"
for cand in ["longitude", "Longitude", "LON"]:
    if "lon" not in ds_main.coords and cand in ds_main.coords:
        rename_coords[cand] = "lon"
if rename_coords:
    ds_main = ds_main.rename(rename_coords)

# Para DataFrame "longo": (date, lat, lon, precip)
df = ds_main.to_dataframe().reset_index()

# Renomeia coluna temporal para 'date'
if "time" in df.columns:
    df = df.rename(columns={"time": "date"})

# Colunas em ordem amigável
cols = [c for c in ["date", "lat", "lon", "precip"] if c in df.columns] + \
       [c for c in df.columns if c not in {"date", "lat", "lon", "precip"}]
df_chuva = df[cols]

# Ordena (opcional)
if SORT_OUTPUT:
    sort_cols = [c for c in ["date", "lat", "lon"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(sort_cols).reset_index(drop=True)


# -------------------- Resumo --------------------
print(f"\n✅ DataFrame criado: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
if "date" in df.columns:
    print(f"   período: {pd.to_datetime(df['date']).min().date()} → {pd.to_datetime(df['date']).max().date()}")
if failed:
    print("\n⚠️ Arquivos com erro:")
    for name, err in failed[:10]:
        print(f" - {name}: {err}")
    if len(failed) > 10:
        print(f"... e mais {len(failed)-10} erros.")


✅ DataFrame criado: 712,400 linhas x 4 colunas
   período: 2015-01-01 → 2017-12-31


### KNN para definir mesorregiões paraenses

In [70]:

USE_PA_BBOX = True
PA_BBOX = dict(lon_min=-58.89833, lon_max=-46.06083,  # Oeste/Leste
               lat_min=-9.84111,  lat_max= 2.59111)   # Sul/Norte

if USE_PA_BBOX:
    df_chuva = df_chuva[
        (df_chuva['lon'] >= PA_BBOX['lon_min']) & (df_chuva['lon'] <= PA_BBOX['lon_max']) &
        (df_chuva['lat'] >= PA_BBOX['lat_min']) & (df_chuva['lat'] <= PA_BBOX['lat_max'])
    ].copy()

# ------------- centróides aproximados das mesorregiões (WGS84) -------------

meso_centroids = pd.DataFrame([
    # name                                lat      lon
    ["Baixo Amazonas (PA)",              -1.94194,  -54.73780],  # região Santarém/Óbidos/Oriximiná
    ["Marajó (PA)",                      -1.68194, -50.48000],  # ilha do Marajó (Soure/Salvaterra)
    ["Metropolitana de Belém (PA)",      -1.45583, -48.50390],  # Belém/Ananindeua
    ["Nordeste Paraense (PA)",           -2.41889, -48.15194],  # Capanema/Bragança/Castanhal eixo
    ["Sudoeste Paraense (PA)",           -3.20278, -52.20583],  # Itaituba/Novo Progresso/Altamira S-O
    ["Sudeste Paraense (PA)",            -6.06778, -49.90194],  # Marabá/Parauapebas/Redenção
], columns=["mesoregion","lat_c","lon_c"])

# ---------------- FUNÇÕES ----------------
def haversine_np(lat1, lon1, lat2, lon2):
    """
    Distância Haversine vetorizada (km).
    """
    R = 6371.0088  # raio médio da Terra em km
    # converte para radianos
    lat1_rad = np.radians(lat1); lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2); lon2_rad = np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def assign_mesoregion_knn(df_points, centroids):
    """
    df_points: DataFrame com colunas ['lat','lon']
    centroids: DataFrame com ['mesoregion','lat_c','lon_c']
    Retorna Series com rótulo da mesorregião (string) por linha de df_points.
    """
    # matriz distâncias: (n_points, n_centroids)
    dists = haversine_np(
        df_points['lat'].to_numpy()[:, None],
        df_points['lon'].to_numpy()[:, None],
        centroids['lat_c'].to_numpy()[None, :],
        centroids['lon_c'].to_numpy()[None, :]
    )
    idx_min = np.argmin(dists, axis=1)
    return pd.Series(centroids['mesoregion'].to_numpy()[idx_min], index=df_points.index)


### Agrupando mesoregião para agregar média de precipitação

In [71]:
coords = (
    df_chuva[['lat','lon']].drop_duplicates().reset_index(drop=True)
)

coords['mesoregion'] = assign_mesoregion_knn(coords, meso_centroids)

for col in ['mesoregion', 'mesoregion_x', 'mesoregion_y']:
    if col in df_chuva.columns:
        df_chuva = df_chuva.drop(columns=col)

# mapeia (lat, lon) -> mesoregion sem usar merge (evita conflitos)
coord_to_meso = { (r.lat, r.lon): r.mesoregion for r in coords.itertuples(index=False) }
df_chuva['mesoregion'] = list(map(coord_to_meso.get, zip(df_chuva['lat'], df_chuva['lon'])))

# --- agregação diária por mesorregião ---
df_meso_daily = (
    df_chuva
    .dropna(subset=['mesoregion'])
    .groupby(['date', 'mesoregion'], as_index=False)
    .agg(precip_mean=('precip', 'mean'))
)


### Agrupando semanas para agregar média de precipitação

Aqui também foi necessário criar um "lag" no dado para que as chuvas de semanas atrás possam se relacionar com o período de colheita.

O valor de lag_k e win_sizes foram definidos a partir de dados dessa pesquisa, que menciona um período de 6-9 meses de chuva para explicar a colheita da safra em questão (https://onlinelibrary.wiley.com/doi/10.1002/jsfa.10164?utm_source=chatgpt.com).

In [72]:
df_meso_daily['date'] = pd.to_datetime(df_meso_daily['date'])

# define semana ISO (início na segunda-feira)
week_start = df_meso_daily['date'] - pd.to_timedelta(df_meso_daily['date'].dt.weekday, unit='D')
df_meso_daily['week_start'] = week_start
df_meso_daily['week_end'] = df_meso_daily['week_start'] + pd.Timedelta(days=6)

# acumulado semanal por mesorregião (mm/semana)
df_meso_weekly = (
    df_meso_daily
    .groupby(['mesoregion', 'week_start', 'week_end'], as_index=False)
    .agg(precip_sum_week_mm=('precip_mean', 'sum'))  # 'precip_mean' está em mm/dia → soma = mm/semana
    .sort_values(['mesoregion', 'week_start'])
    .reset_index(drop=True)
)
# --- parâmetros ---
LAG_K = 36       
WIN_SIZES = [8]  

# --- features defasadas (sem vazamento) ---
df_feat = (
    df_meso_weekly
    .sort_values(['mesoregion','week_start'])
    .copy()
)

# lag simples (semana exata K atrás)
df_feat[f'precip_sum_lag_{LAG_K}w'] = (
    df_feat.groupby('mesoregion')['precip_sum_week_mm'].shift(LAG_K)
)

# acumulados: soma das últimas W semanas, terminando K semanas antes
for W in WIN_SIZES:
    df_feat[f'precip_rollsum_{W}w_end_lag_{LAG_K}w'] = (
        df_feat.groupby('mesoregion')['precip_sum_week_mm']
              .apply(lambda s: s.shift(LAG_K).rolling(W, min_periods=1).sum())
              .reset_index(level=0, drop=True)
    )

# mantém só chave + features (evita usar chuva da semana corrente)
df_feat = df_feat[['mesoregion','week_start'] + [c for c in df_feat.columns if c.startswith('precip_')]]

# --- corte dinâmico para evitar NaN por falta de histórico ---
weeks_needed = LAG_K + (max(WIN_SIZES) if WIN_SIZES else 0) - 1
cutoff = df_meso_weekly['week_start'].min() + pd.Timedelta(weeks=max(0, weeks_needed))
df_meso_weekly = df_feat[df_feat['week_start'] >= cutoff].reset_index(drop=True)

cutoff = pd.Timestamp('2016-01-01') 
df_meso_weekly = (
    df_meso_weekly[df_meso_weekly['week_start'] >= cutoff]
    .reset_index(drop=True)
)



In [73]:
print(df_meso_weekly.tail())

                 mesoregion week_start  precip_sum_week_mm  \
619  Sudoeste Paraense (PA) 2017-11-27           14.480295   
620  Sudoeste Paraense (PA) 2017-12-04           18.697744   
621  Sudoeste Paraense (PA) 2017-12-11           85.864632   
622  Sudoeste Paraense (PA) 2017-12-18           38.662868   
623  Sudoeste Paraense (PA) 2017-12-25           51.629360   

     precip_sum_lag_36w  precip_rollsum_8w_end_lag_36w  
619           66.149475                     634.279564  
620           68.375801                     594.073883  
621           63.364788                     587.023556  
622           85.780838                     581.105656  
623           47.514755                     529.973698  


In [74]:
acai_dataset = pd.read_excel("./Acai_Datasets/Lavoura_Permanente_Acai_Para.xlsx",  sheet_name='Quantidade produzida (Tonela...',header=None)
acai_dataset

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Tabela 1613 - Área destinada à colheita, área ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Variável - Quantidade produzida (Toneladas),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nível,Cód.,Unidade da Federação e Mesorregião Geográfica,Ano x Produto das lavouras permanentes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2016,2017,2018,2019,2020,2021,2022,2023,2024
4,NaN,NaN,NaN,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí,Açaí
5,UF,11,Rondônia,-,1152,1858,2242,2260,1441,2115,2664,2629
6,UF,12,Acre,-,-,-,-,-,200,20,181,129
7,UF,13,Amazonas,9576,52785,62329,67757,73538,83321,90616,105211,91345
8,UF,14,Roraima,851,3513,3449,4153,4271,1957,2749,3087,3222
9,UF,15,Pará,1080612,1274056,1230699,1320150,1389941,1388116,1595455,1576302,1612142


## Tratamento dos Dados

 Necessário converter os dados relacionados a açaí para o formato Série Temporal (por semana).

In [89]:
def make_training_data_from_single(acai_dataset: pd.DataFrame, years=None) -> pd.DataFrame:
    """
    Extrai (mesoregion, ano, toneladas) do sheet 'estilo IBGE' onde os anos estão em uma linha
    e os valores anuais em colunas. Detecta automaticamente a linha de anos.
    Retorna um DF long com ['mesoregion','ano','toneladas'].
    """

    # 1) Encontrar a linha que contém mais "anos" (inteiros 1800..2100)
    def _count_years(row: pd.Series) -> int:
        nums = pd.to_numeric(row, errors="coerce")
        return int(((nums >= 1800) & (nums <= 2100)).sum())

    year_row_idx = acai_dataset.apply(_count_years, axis=1).idxmax()

    # 2) Construir mapa coluna->ano usando a linha encontrada
    year_map = {}
    for c in acai_dataset.columns:
        val = acai_dataset.iloc[year_row_idx, c]
        val_num = pd.to_numeric(val, errors="coerce")
        if pd.notna(val_num):
            yn = int(val_num)
            if 1800 <= yn <= 2100:
                year_map[c] = yn

    if not year_map:
        raise ValueError("Não encontrei uma linha com anos no acai_dataset.")

    year_cols = sorted(year_map.keys(), key=lambda k: year_map[k])

    # 3) Filtrar mesorregiões (linhas com 0 == 'ME' e nome em col 2)
    mask_me = acai_dataset[0].astype(str).eq("ME") & acai_dataset[2].notna()
    df_me = acai_dataset.loc[mask_me, [2] + year_cols].copy()

    # 4) Wide -> long
    long = df_me.melt(id_vars=[2], value_vars=year_cols,
                      var_name="col_idx", value_name="toneladas")

    # 5) Mapear coluna para ano e limpar
    long["ano"] = long["col_idx"].map(year_map).astype(int)
    long = long.drop(columns=["col_idx"]).rename(columns={2: "mesoregion"})
    long["mesoregion"] = long["mesoregion"].astype(str).str.strip()
    long["toneladas"] = pd.to_numeric(long["toneladas"], errors="coerce")

    # 6) Filtrar anos desejados (opcional)
    if years is not None:
        years = set(int(y) for y in years)
        long = long[long["ano"].isin(years)]

    # 7) Final
    return (
        long[["mesoregion", "ano", "toneladas"]]
        .dropna(subset=["toneladas"])
        .reset_index(drop=True)
    )



training_data_df = make_training_data_from_single(acai_dataset, years=[2016, 2017])
print(training_data_df.tail())

                     mesoregion   ano  toneladas
7                   Marajó (PA)  2017     324528
8   Metropolitana de Belém (PA)  2017     185650
9        Nordeste Paraense (PA)  2017     691025
10       Sudoeste Paraense (PA)  2017       4821
11        Sudeste Paraense (PA)  2017      54796


Agora os dados serão convertidos para um modelo de semana em semana, utilizando como referência os seguintes valores de distribuição de produção (em toneladas)

- Mensal (BRS Pará, sem irrigação): Jan 15, Fev 10, Mar 3, Abr 2, Mai 0, Jun 0, Jul 0, Ago 3, Set 7, Out 15, Nov 25, Dez 20. (https://www.infoteca.cnptia.embrapa.br/infoteca/bitstream/doc/1101575/1/SistemadeproducaoAcai2018.pdf)
- Semestres (Pará, sem irrigação): S1 (jan–jun) ~20–30% · S2 (jul–dez) ~70–80%.  https://www.alice.cnptia.embrapa.br/bitstream/doc/994953/1/CULTIVO20.pdf

In [90]:
MONTH_WEIGHTS = {1:0.15,2:0.10,3:0.03,4:0.02,5:0.00,6:0.00,7:0.00,8:0.03,9:0.07,10:0.15,11:0.25,12:0.20}

def mondays_in_month(year: int, month: int) -> pd.DatetimeIndex:
    start = pd.Timestamp(year=year, month=month, day=1)
    end = pd.Timestamp(year=year, month=month, day=monthrange(year, month)[1])
    d = pd.date_range(start, end, freq="W-MON")
    if start.weekday() == 0 and (len(d) == 0 or d[0] != start):
        d = d.insert(0, start)
    return d[d.month == month]

def generate_weekly_series_meso(df_meso: pd.DataFrame, seed: int = 42) -> pd.DataFrame:
    """
    df_meso: colunas ['mesoregion','ano','toneladas'] (um total anual por mesorregião)
    retorna: semanal com ['mesoregion','date','year','month','week','toneladas_semana','dist_ano','dist_mes','periodo']
    """
    rng = np.random.default_rng(seed)
    if not np.isclose(sum(MONTH_WEIGHTS[m] for m in range(1,7)), 0.30) or \
       not np.isclose(sum(MONTH_WEIGHTS[m] for m in range(7,13)), 0.70):
        raise ValueError("MONTH_WEIGHTS deve somar 30% (jan-jun) e 70% (jul-dez).")

    rows = []
    for _, r in df_meso.iterrows():
        reg  = str(r['mesoregion'])
        year = int(r['ano'])
        total_year = float(r['toneladas'])

        for m in range(1,13):
            w = MONTH_WEIGHTS[m]
            month_total = total_year * w
            weeks = mondays_in_month(year, m)
            n = len(weeks)
            if n == 0: 
                continue
            alpha = 5.0
            shares = np.zeros(n) if w == 0.0 else rng.dirichlet(np.full(n, alpha))
            tons = month_total * shares
            for d, t in zip(weeks, tons):
                rows.append({
                    'mesoregion': reg,
                    'date': d.normalize(),
                    'year': year,
                    'month': m,
                    'week': int(d.isocalendar().week),
                    'toneladas_semana': float(t),
                    'dist_ano': float(t / total_year if total_year > 0 else 0.0),
                    'dist_mes': float(t / month_total) if month_total > 0 else (0.0 if w == 0 else np.nan),
                    'periodo': 'entressafra' if m <= 6 else 'safra'
                })
    return pd.DataFrame(rows).sort_values(['mesoregion','year','date']).reset_index(drop=True)


weekly_series_meso = generate_weekly_series_meso(training_data_df, seed=123)
weekly_series_meso

,mesoregion,date,year,month,week,toneladas_semana,dist_ano,dist_mes,periodo
0,Baixo Amazonas (PA),2016-01-04,2016,1,1,197.100086,0.020043,0.133618,entressafra
1,Baixo Amazonas (PA),2016-01-11,2016,1,2,557.988023,0.056741,0.378271,entressafra
2,Baixo Amazonas (PA),2016-01-18,2016,1,3,482.449373,0.049059,0.327062,entressafra
3,Baixo Amazonas (PA),2016-01-25,2016,1,4,237.562519,0.024157,0.161048,entressafra
4,Baixo Amazonas (PA),2016-02-01,2016,2,5,120.990118,0.012303,0.123032,entressafra
...,...,...,...,...,...,...,...,...,...
619,Sudoeste Paraense (PA),2017-11-27,2017,11,48,183.214121,0.038003,0.152013,safra
620,Sudoeste Paraense (PA),2017-12-04,2017,12,49,283.288938,0.058761,0.293807,safra
621,Sudoeste Paraense (PA),2017-12-11,2017,12,50,160.122102,0.033213,0.166067,safra
622,Sudoeste Paraense (PA),2017-12-18,2017,12,51,224.190647,0.046503,0.232515,safra


### Concatenação dos dados de precipitação e dos dados de produção

In [91]:

# 1) chave semanal = segunda-feira (week_start)
df_meso_weekly['week_start'] = pd.to_datetime(df_meso_weekly['week_start'])
df_meso_weekly['mesoregion'] = df_meso_weekly['mesoregion'].astype(str).str.strip()

weekly_series_meso['date'] = pd.to_datetime(weekly_series_meso['date'])
weekly_series_meso['mesoregion'] = weekly_series_meso['mesoregion'].astype(str).str.strip()
wk = weekly_series_meso.rename(columns={'date': 'week_start'}).copy()


# 2) merge limpo por mesorregião + semana (agora levando TODAS as colunas precip_ de df_meso_weekly)
precip_cols = [c for c in df_meso_weekly.columns if c.startswith('precip_')]
df_weekly_join = (
    wk.merge(
        df_meso_weekly[['mesoregion', 'week_start'] + precip_cols],
        on=['mesoregion', 'week_start'],
        how='left',
        validate='one_to_one'
    )
    .sort_values(['mesoregion', 'week_start'])
    .reset_index(drop=True)
)

df_weekly_join['week_end'] = df_weekly_join['week_start'] + pd.Timedelta(days=6)


base_cols = ['mesoregion','week_start','week_end','year','month','week',
             'toneladas_semana','periodo']
training_data = df_weekly_join[base_cols + precip_cols]


training_data = training_data[training_data['mesoregion'] == 'Nordeste Paraense (PA)']
training_data 

,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
312,Nordeste Paraense (PA),2016-01-04,2016-01-10,2016,1,1,27771.151243,entressafra,25.741644,66.697495,641.472336
313,Nordeste Paraense (PA),2016-01-11,2016-01-17,2016,1,2,45189.561815,entressafra,53.002594,65.546700,660.171753
314,Nordeste Paraense (PA),2016-01-18,2016-01-24,2016,1,3,20887.984346,entressafra,28.687334,8.286710,537.255216
315,Nordeste Paraense (PA),2016-01-25,2016-01-31,2016,1,4,11277.602596,entressafra,100.139969,35.914242,498.886483
316,Nordeste Paraense (PA),2016-02-01,2016-02-07,2016,2,5,16964.410896,entressafra,91.146965,21.524551,453.555245
...,...,...,...,...,...,...,...,...,...,...,...
411,Nordeste Paraense (PA),2017-11-27,2017-12-03,2017,11,48,27974.772963,safra,12.195934,92.854034,836.052078
412,Nordeste Paraense (PA),2017-12-04,2017-12-10,2017,12,49,28764.775709,safra,7.283447,70.910820,831.695145
413,Nordeste Paraense (PA),2017-12-11,2017-12-17,2017,12,50,28792.462132,safra,56.669525,79.253830,833.102669
414,Nordeste Paraense (PA),2017-12-18,2017-12-24,2017,12,51,52276.552824,safra,99.665421,84.343636,762.984421


## Análise dos Dados

É possível aferir sobre a precipitação observando a média, máxima e mediana, que existe uma forte assimetria, destacando que algumas semanas são responsáveis por boa parte da precipitação do trimestre.


In [92]:
training_data.describe(include="all")

,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
count,104,104,104,104.000000,104.000000,104.000000,104.000000,104,104.000000,104.000000,104.000000
unique,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
top,Nordeste Paraense (PA),NaN,NaN,NaN,NaN,NaN,NaN,entressafra,NaN,NaN,NaN
freq,104,NaN,NaN,NaN,NaN,NaN,NaN,52,NaN,NaN,NaN
mean,NaN,2016-12-29 12:00:00,2017-01-04 12:00:00,2016.500000,6.461538,26.500000,13383.336538,NaN,38.954868,36.872456,297.767503
min,NaN,2016-01-04 00:00:00,2016-01-10 00:00:00,2016.000000,1.000000,1.000000,0.000000,NaN,0.309547,0.089477,20.798080
25%,NaN,2016-07-02 06:00:00,2016-07-08 06:00:00,2016.000000,3.750000,13.750000,0.000000,NaN,7.209070,5.797648,73.261050
50%,NaN,2016-12-29 12:00:00,2017-01-04 12:00:00,2016.500000,6.500000,26.500000,7796.527740,NaN,21.559301,22.062949,175.701795
75%,NaN,2017-06-27 18:00:00,2017-07-03 18:00:00,2017.000000,9.250000,39.250000,21299.569812,NaN,72.485437,65.834398,514.250110
max,NaN,2017-12-25 00:00:00,2017-12-31 00:00:00,2017.000000,12.000000,52.000000,74980.391465,NaN,154.461884,154.461884,876.992096


In [93]:
training_data.corr(numeric_only = True)

,year,month,week,toneladas_semana,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
year,1.000000e+00,-0.005601,-1.415244e-13,-0.006221,0.087115,0.162792,0.169944
month,-5.601384e-03,1.000000,9.964929e-01,0.405501,-0.555362,0.597449,0.434362
week,-1.415244e-13,0.996493,1.000000e+00,0.401297,-0.552327,0.596862,0.436019
toneladas_semana,-6.220719e-03,0.405501,4.012968e-01,1.000000,0.073532,0.630689,0.823329
precip_sum_week_mm,8.711489e-02,-0.555362,-5.523270e-01,0.073532,1.000000,-0.227545,0.036427
precip_sum_lag_36w,1.627915e-01,0.597449,5.968623e-01,0.630689,-0.227545,1.000000,0.802396
precip_rollsum_8w_end_lag_36w,1.699444e-01,0.434362,4.360188e-01,0.823329,0.036427,0.802396,1.000000


Maior quantidade de chuvas no começo e final do ano, uma menor quatidade de chuva ao meio do ano, ainda que dificilmente chegue em 0 de fato.

In [94]:
fig = px.scatter_matrix(
    training_data,
    dimensions=['precip_rollsum_8w_end_lag_36w', 'week'],
    color='mesoregion'
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

Existe uma maior produção de açaí no começo e final do ano. Meio do ano costuma apresentar menos açaí.

In [126]:
fig = px.scatter_matrix(
    training_data,
    dimensions=['toneladas_semana', 'week'],
    color='mesoregion'
)
fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

É interessante notar que mais chuvas não significa necessariamente mais toneladas a serem produzidas, e sim que existe uma certa quantidade de chuva ideal para a produção de açaí.

Como existem muitos outliers, provavelmente o MSE será a fórmula utilizada para métrica de erros.

In [96]:

fig = px.scatter_matrix(
    training_data,
    dimensions=['precip_rollsum_8w_end_lag_36w', 'toneladas_semana'],
    color='mesoregion'
)

fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

## Remoção dos Outliers

In [97]:
def remove_outliers_mad(
    df: pd.DataFrame,
    col: str = "toneladas_semana",
    by: list = ("mesoregion",),          # agrupa (mude se quiser)
    thresh: float = 3.5,                 # >3.5 z-MAD costuma ser outlier
    return_outliers: bool = True
):
    def _flag(g):
        x = g[col].astype(float)
        med = np.median(x)
        mad = 1.4826 * np.median(np.abs(x - med))  # MAD robust
        zmad = (x - med) / (mad + 1e-9)
        g["_is_outlier"] = np.abs(zmad) > thresh
        return g

    df2 = df.copy()
    df2 = df2.groupby(list(by), dropna=False, group_keys=False).apply(_flag)

    clean = df2.loc[~df2["_is_outlier"]].drop(columns=["_is_outlier"])
    outs  = df2.loc[df2["_is_outlier"]].drop(columns=["_is_outlier"])
    return (clean, outs) if return_outliers else clean


df, df_out = remove_outliers_mad(training_data, col="toneladas_semana", by=["mesoregion"], thresh=2.5)
df.describe(include="all")



C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7336\3039602817.py:17: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,mesoregion,week_start,week_end,year,month,week,toneladas_semana,periodo,precip_sum_week_mm,precip_sum_lag_36w,precip_rollsum_8w_end_lag_36w
count,94,94,94,94.000000,94.000000,94.000000,94.000000,94,94.000000,94.000000,94.000000
unique,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
top,Nordeste Paraense (PA),NaN,NaN,NaN,NaN,NaN,NaN,entressafra,NaN,NaN,NaN
freq,94,NaN,NaN,NaN,NaN,NaN,NaN,50,NaN,NaN,NaN
mean,NaN,2016-12-24 06:53:37.021276672,2016-12-30 06:53:37.021276672,2016.510638,6.159574,25.202128,9729.558079,NaN,38.371056,32.945065,257.823686
min,NaN,2016-01-04 00:00:00,2016-01-10 00:00:00,2016.000000,1.000000,1.000000,0.000000,NaN,0.309547,0.089477,20.798080
25%,NaN,2016-06-21 18:00:00,2016-06-27 18:00:00,2016.000000,3.250000,13.250000,0.000000,NaN,6.955577,5.447764,64.042751
50%,NaN,2017-01-12 12:00:00,2017-01-18 12:00:00,2017.000000,6.000000,25.000000,5487.812219,NaN,19.213753,17.141375,138.560111
75%,NaN,2017-06-24 06:00:00,2017-06-30 06:00:00,2017.000000,9.000000,36.750000,16808.816082,NaN,71.396204,53.086239,416.891982
max,NaN,2017-12-25 00:00:00,2017-12-31 00:00:00,2017.000000,12.000000,52.000000,32382.181514,NaN,154.461884,154.461884,876.992096


In [125]:
fig = px.scatter_matrix(
    df,
    dimensions=['precip_rollsum_8w_end_lag_36w', 'toneladas_semana'],
    color='periodo'
)

fig.update_traces(diagonal_visible=False)
fig.update_layout(height=600, width=800)
fig.show()

## Desenvolvimento da Machine Learning

A técnica a ser utilizada inicialmente será uma regressão linear

### Regressores

In [99]:
class AcaiRegressor:
    def __init__(
        self,
        technique: Literal["linear", "poly2"] = "linear",
        feature_mode: Literal["engineered", "explicit"] = "engineered",
        num_cols: Optional[list] = None,     # colunas numéricas do df
        cat_cols: Optional[list] = None,     # colunas categóricas do df (one-hot)
        cyclic_cols: Optional[dict] = None,  # ex.: {"week": 52, "month": 12}
        # splits
        val_frac: float = 0.2,
        test_frac: float = 0.2,
        # model & train hparams
        optimizer: str = "adam",
        learning_rate: float = 1e-3,
        l2: float = 0.0,                 # L2 na camada linear
        epochs: int = 300,
        batch_size: Optional[int] = 32,
        early_stopping_patience: int = 80,
        early_min_delta: float = 1e-3,
        # feature flags
        use_region_ohe: bool = True,
        include_harmonic_k2: bool = True,
        include_month_cycle: bool = True,
        include_interactions: bool = True,  # precip × seno/cosseno
        loss_fn: Literal["mse", "mae"] = "mse",
        feature_spec: Optional[dict] = None,
        passthrough_cols: Optional[list] = None,
    ):


        self.technique = technique
        self.val_frac = val_frac
        self.test_frac = test_frac
        self.optimizer_name = optimizer.lower()
        self.learning_rate = learning_rate
        self.l2 = l2
        self.epochs = epochs
        self.batch_size = batch_size
        self.early_patience = early_stopping_patience
        self.early_min_delta = early_min_delta
        self.use_region_ohe = use_region_ohe
        self.include_harmonic_k2 = include_harmonic_k2
        self.include_month_cycle = include_month_cycle
        self.include_interactions = include_interactions

        # serão definidos após fit()
        self.ohe_: Optional[OneHotEncoder] = None
        self.xsc_: Optional[StandardScaler] = None
        self.ysc_: Optional[StandardScaler] = None
        self.model_: Optional[keras.Model] = None
        self.history_ = None
        self.feature_names_: Optional[list] = None
        self.splits_: Dict[str, Any] = {}

        self.feature_mode = feature_mode
        self.num_cols = num_cols or []
        self.cat_cols = cat_cols or []
        self.cyclic_cols = cyclic_cols or {}   # nome->período (int)
        self.loss_fn = loss_fn.lower()


    # ---------- feature engineering ----------
    def _build_numeric_features(self, d: pd.DataFrame) -> np.ndarray:
        names, cols = [], []
    
        if self.feature_mode == "explicit":
            # (a) passa colunas numéricas cruas
            for c in self.num_cols:
                if c in d.columns:
                    v = pd.to_numeric(d[c], errors="coerce").to_numpy()
                    cols += [v]; names += [c]
            # (b) expansões cíclicas opcionais (sin/cos) sobre colunas listadas
            for c, period in self.cyclic_cols.items():
                if c in d.columns:
                    v = pd.to_numeric(d[c], errors="coerce").to_numpy().astype(float)
                    s = np.sin(2*np.pi*v/float(period))
                    c_ = np.cos(2*np.pi*v/float(period))
                    cols += [s, c_]; names += [f"{c}_sin", f"{c}_cos"]
            X_num = np.column_stack(cols) if cols else np.empty((len(d), 0))
            self.feature_names_ = names
            return X_num
    
        # === modo "engineered" (seu comportamento anterior) ===
        # básicos
        week = d["week"].astype(float).to_numpy()
        denom_w = 52.0
        sin_w = np.sin(2*np.pi*week/denom_w)
        cos_w = np.cos(2*np.pi*week/denom_w)
        p = d["precip_rollsum_8w_end_lag_36w"].astype(float).to_numpy()
        p = np.log1p(p)
    
        cols += [p, sin_w, cos_w]; names += ["p", "sin_w", "cos_w"]
    
        if self.include_month_cycle:
            month = d["month"].astype(float).to_numpy()
            sin_m = np.sin(2*np.pi*month/12.0)
            cos_m = np.cos(2*np.pi*month/12.0)
            cols += [sin_m, cos_m]; names += ["sin_m", "cos_m"]
    
        if self.technique == "poly2":
            cols += [p**2]; names += ["p2"]
            if self.include_interactions:
                cols += [p*sin_w, p*cos_w]; names += ["p_sin_w", "p_cos_w"]
                if self.include_month_cycle:
                    cols += [p*sin_m, p*cos_m]; names += ["p_sin_m", "p_cos_m"]
            if self.include_harmonic_k2:
                sin2w = np.sin(2*np.pi*2*week/denom_w)
                cos2w = np.cos(2*np.pi*2*week/denom_w)
                cols += [sin2w, cos2w]; names += ["sin2w", "cos2w"]
    
        X_num = np.column_stack(cols)
        self.feature_names_ = names
        return X_num


    def _build_X(self, df_train: pd.DataFrame, df_other: pd.DataFrame):
        """
        Monta X_train e X_other:
          - numéricas: vindas do _build_numeric_features(...)
          - categóricas: OHE genérico (todas as colunas informadas) com handle_unknown='ignore'
          - escala: StandardScaler fitado em X_train
        Também popula self.feature_names_full_ (numéricas + nomes OHE).
        """
        # --- 1) Numéricas ---
        Xtr_num = self._build_numeric_features(df_train)   # define self.feature_names_ (numéricas)
        Xot_num = self._build_numeric_features(df_other)
    
        # --- 2) Quais colunas categóricas serão usadas ---
        if self.feature_mode == "explicit":
            cat_cols = [c for c in (self.cat_cols or []) if c in df_train.columns]
        else:  # engineered
            cat_cols = ["mesoregion"] if (getattr(self, "use_region_ohe", True) and "mesoregion" in df_train.columns) else []
    
        # --- 3) One-Hot (genérico, para N colunas categóricas) ---
        ohe_feature_names = []
        if len(cat_cols) > 0:
            # compatibilidade: sklearn<1.2 usa 'sparse', >=1.2 tem 'sparse_output'
            try:
                self.ohe_ = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=float)
            except TypeError:
                self.ohe_ = OneHotEncoder(handle_unknown="ignore", sparse=False, dtype=float)
    
            Ztr = self.ohe_.fit_transform(df_train[cat_cols])
            Zot = self.ohe_.transform(df_other[cat_cols])
    
            # nomes das dummies: <col>=<categoria>
            for col, cats in zip(cat_cols, self.ohe_.categories_):
                ohe_feature_names += [f"{col}={cat}" for cat in cats]
    
        else:
            # sem categóricas -> matrizes vazias com número de linhas correto
            Ztr = np.empty((len(df_train), 0), dtype=float)
            Zot = np.empty((len(df_other), 0), dtype=float)
    
        # --- 4) Concatena numéricas + OHE ---
        Xtr = np.hstack([Xtr_num, Ztr])
        Xot = np.hstack([Xot_num, Zot])
    
        # guarda nomes completos (para debug/feature importance fora de redes etc.)
        num_names = list(self.feature_names_ or [])
        self.feature_names_full_ = num_names + ohe_feature_names
    
        # --- 5) Escala (fit só no treino) ---
        self.xsc_ = StandardScaler().fit(Xtr)
        return self.xsc_.transform(Xtr), self.xsc_.transform(Xot)
    
        


    # ---------- split temporal ----------
    def _temporal_split(self, df: pd.DataFrame):
        n = len(df)
        cut_test = int(n * (1 - self.test_frac))
        cut_val  = int(cut_test * (1 - self.val_frac))
        df_train = df.iloc[:cut_val].copy()
        df_val   = df.iloc[cut_val:cut_test].copy()
        df_test  = df.iloc[cut_test:].copy()
    
        # y
        y_train = df_train["toneladas_semana"].astype(float).to_numpy()
        y_val   = df_val["toneladas_semana"].astype(float).to_numpy()
        y_test  = df_test["toneladas_semana"].astype(float).to_numpy()
    
        # X (numéricas + OHE, escaladas) para train/val
        X_train, X_val = self._build_X(df_train, df_val)
    
        # ---------- TESTE: mesma lógica de OHE usada em _build_X/predict ----------
        X_test_num = self._build_numeric_features(df_test)
    
        use_ohe = False
        cols_test = None
        if self.feature_mode == "explicit":
            if len(self.cat_cols) > 0:
                use_ohe = True
                cols_test = self.cat_cols
        else:  # engineered
            if getattr(self, "use_region_ohe", True):
                use_ohe = True
                cols_test = ["mesoregion"]
    
        if use_ohe:
            # self.ohe_ já foi fitado em _build_X (no train)
            Ztest = self.ohe_.transform(df_test[cols_test])
            X_test = np.hstack([X_test_num, Ztest])
        else:
            X_test = X_test_num
    
        # escala de X (self.xsc_ já foi fitado em _build_X com X_train)
        X_test = self.xsc_.transform(X_test)
    
        # scaler de y (fit só no treino)
        self.ysc_ = StandardScaler().fit(y_train.reshape(-1,1))
        y_train_s = self.ysc_.transform(y_train.reshape(-1,1)).ravel()
        y_val_s   = self.ysc_.transform(y_val.reshape(-1,1)).ravel()
    
        self.splits_ = dict(
            df_train=df_train, df_val=df_val, df_test=df_test,
            X_train=X_train, y_train=y_train, y_train_s=y_train_s,
            X_val=X_val, y_val=y_val, y_val_s=y_val_s,
            X_test=X_test, y_test=y_test
        )
    

    def _build_model(self, n_features: int) -> keras.Model:
        reg = regularizers.l2(self.l2) if self.l2 > 0 else None
        inputs = keras.Input(shape=(n_features,))
        outputs = layers.Dense(1, activation="linear", use_bias=True,
                               kernel_regularizer=reg, name="linreg")(inputs)
        model = keras.Model(inputs, outputs)
    
        if self.optimizer_name == "adam":
            opt = keras.optimizers.Adam(self.learning_rate)
        elif self.optimizer_name == "sgd":
            opt = keras.optimizers.SGD(self.learning_rate, momentum=0.9, nesterov=True)
        else:
            raise ValueError("optimizer deve ser 'adam' ou 'sgd'")
    
        # NOVO: escolha da loss e métricas adicionais para monitorar
        loss_name = "mse" if self.loss_fn == "mse" else "mae"
        model.compile(
            optimizer=opt,
            loss=loss_name,
            metrics=[
                keras.metrics.MeanAbsoluteError(name="mae"),
                keras.metrics.MeanSquaredError(name="mse"),
            ],
        )
        return model


        # ---------- API pública ----------
    def fit(self, df: pd.DataFrame):
        """
        df precisa conter: 'week_start', 'toneladas_semana' e as colunas listadas
        em num_cols, cat_cols e cyclic_cols (apenas essas serão usadas).
        """
        # monta o conjunto mínimo de colunas obrigatórias
        req_cols = set(self.num_cols) | set(self.cat_cols) | set(self.cyclic_cols.keys())
        req_cols |= {"toneladas_semana", "week_start"}  # alvo + ordem temporal
    
        # checagem simples
        missing = [c for c in req_cols if c not in df.columns]
        if missing:
            raise ValueError(f"Colunas ausentes no df para o modo 'explicit': {missing}")
    
        # filtra o DF e remove NaNs só nas colunas necessárias
        df = df.loc[:, list(req_cols)].dropna(subset=list(req_cols)).sort_values("week_start").copy()
    
        self._temporal_split(df)
        Xtr, ytr_s = self.splits_["X_train"], self.splits_["y_train_s"]
        Xval, yval_s = self.splits_["X_val"], self.splits_["y_val_s"]
    
        if self.batch_size is None:
            self.batch_size = min(32, max(8, int(len(Xtr)/10)))
    
        self.model_ = self._build_model(Xtr.shape[1])
    
        es = keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=self.early_patience, min_delta=self.early_min_delta,
            restore_best_weights=True, verbose=1
        )
    
        print(f"\n[DEBUG] n_features: {Xtr.shape[1]}")
        print("[DEBUG] features numéricas:", self.feature_names_)
        if hasattr(self, "ohe_") and self.ohe_ is not None:
            print("[DEBUG] categorias OHE:", [list(c) for c in self.ohe_.categories_])
    
        self.history_ = self.model_.fit(
            Xtr, ytr_s,
            validation_data=(Xval, yval_s),
            epochs=self.epochs,
            batch_size=self.batch_size,
            shuffle=False,
            callbacks=[es],
            verbose=1
        )
        return self


    # --- troque o método predict inteiro por este ---
    def predict(self, df_new: pd.DataFrame) -> np.ndarray:
        X_num = self._build_numeric_features(df_new)
    
        # mesma decisão de OHE usada no _build_X
        use_ohe = False
        cols_infer = None
        if self.feature_mode == "explicit":
            if len(self.cat_cols) > 0:
                use_ohe = True
                cols_infer = self.cat_cols
        else:
            if getattr(self, "use_region_ohe", True):
                use_ohe = True
                cols_infer = ["mesoregion"]
    
        if use_ohe:
            Z = self.ohe_.transform(df_new[cols_infer])
            X = np.hstack([X_num, Z])
        else:
            X = X_num
    
        Xs = self.xsc_.transform(X)
        y_s = self.model_.predict(Xs, verbose=0).ravel()
        return self.ysc_.inverse_transform(y_s.reshape(-1,1)).ravel()


    def _metrics(self, y_true, y_hat) -> Dict[str, float]:
        mse  = mean_squared_error(y_true, y_hat)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(y_true, y_hat)
        smape = float(np.mean(
            2.0 * np.abs(y_hat - y_true) / (np.abs(y_true) + np.abs(y_hat) + 1e-9)
        ))
        r2   = r2_score(y_true, y_hat)
        nrmse_std = rmse / (np.std(y_true) + 1e-9)
        nrmse_rng = rmse / (np.max(y_true) - np.min(y_true) + 1e-9)
        return dict(MSE=mse, RMSE=rmse, MAE=mae, SMAPE=smape, R2=r2,
                    NRMSE_std=nrmse_std, NRMSE_range=nrmse_rng)

    def evaluate(self) -> Dict[str, Dict[str, float]]:
        """Retorna métricas para TREINO, VALIDAÇÃO e TESTE (em toneladas)."""
        Xtr = self.splits_["X_train"]; ytr_true = self.splits_["y_train"]
        Xval = self.splits_["X_val"];  yval_true = self.splits_["y_val"]
        Xte = self.splits_["X_test"];  yte_true = self.splits_["y_test"]
    
        # preds -> toneladas
        ytr_pred = self.ysc_.inverse_transform(self.model_.predict(Xtr, verbose=0).ravel().reshape(-1,1)).ravel()
        yval_pred = self.ysc_.inverse_transform(self.model_.predict(Xval, verbose=0).ravel().reshape(-1,1)).ravel()
        yte_pred  = self.ysc_.inverse_transform(self.model_.predict(Xte,  verbose=0).ravel().reshape(-1,1)).ravel()
    
        # acurácia dentro de ±10%
        def acc_within(y_true, y_hat, tol=0.1):
            rel = np.abs(y_hat - y_true) / (np.abs(y_true) + 1e-9)
            return float((rel <= tol).mean())
    
        return dict(
            TREINO={**self._metrics(ytr_true, ytr_pred), "Acc@10%": acc_within(ytr_true, ytr_pred)},
            VALIDACAO={**self._metrics(yval_true, yval_pred), "Acc@10%": acc_within(yval_true, yval_pred)},
            TESTE={**self._metrics(yte_true, yte_pred), "Acc@10%": acc_within(yte_true, yte_pred), "Acc@20%": acc_within(yte_true, yte_pred, tol=0.2),
                    "Acc@30%": acc_within(yte_true, yte_pred, tol=0.3)},
        )

            # ======= helpers internos =======
    def _get_split_df_(self, split:str):
        key = dict(TREINO="df_train", VALIDACAO="df_val", TESTE="df_test")[split.upper()]
        return self.splits_[key].copy()
    
    def _split_arrays_(self, split:str):
        s = self.splits_
        if split.upper()=="TREINO":
            X, y = s["X_train"], s["y_train"]
        elif split.upper()=="VALIDACAO":
            X, y = s["X_val"], s["y_val"]
        else:
            X, y = s["X_test"], s["y_test"]
        # previsão em escala original (toneladas)
        y_hat_s = self.model_.predict(X, verbose=0).ravel()
        y_hat   = self.ysc_.inverse_transform(y_hat_s.reshape(-1,1)).ravel()
        return y, y_hat
    
    def _mode_or_first_(self, series: pd.Series):
        try:
            return series.mode(dropna=True).iloc[0]
        except Exception:
            return series.dropna().iloc[0] if series.dropna().shape[0] else None

    def plot_pairs_real_pred(self, split: str = "TESTE", x_col: str = "week_start", tol: float = 0.10):

        # --- pega df do split e prepara X ---
        df_s = self._get_split_df_(split).copy()
        if x_col not in df_s.columns:
            raise ValueError(f"x_col '{x_col}' não está no df do split '{split}'.")
        df_s = df_s.sort_values(x_col)
    
        # eixo X
        if x_col == "week_start":
            x_vals = pd.to_datetime(df_s[x_col])
            x_title = "tempo (week_start)"
        else:
            x_vals = pd.to_numeric(df_s[x_col], errors="coerce")
            x_title = x_col
    
        # y real e previsto
        y_true = pd.to_numeric(df_s["toneladas_semana"], errors="coerce").to_numpy(dtype=float)
        y_pred = self.predict(df_s).astype(float)


        mask = np.isfinite(y_true) & np.isfinite(y_pred)
        y_true = y_true[mask]
        y_pred = y_pred[mask]
        ape    = np.abs(y_pred - y_true) / (np.abs(y_true) + 1e-9)
        acc10  = float((ape <= tol).mean()) * 100.0 

    
        # métricas ponto a ponto
        err   = y_pred - y_true
        ape   = np.abs(err) / (np.abs(y_true) + 1e-9)
        acc10 = float((ape <= tol).mean()) * 100.0
    
        # paleta (repetida se necessário) e cor por amostra
        palette = [
            "#636EFA","#EF553B","#00CC96","#AB63FA","#FFA15A",
            "#19D3F3","#FF6692","#B6E880","#FF97FF","#FECB52"
        ]
        colors = [palette[i % len(palette)] for i in range(len(df_s))]
    
        fig = go.Figure()
    
        # segmentos conectando real -> previsto (um por amostra, para manter cor por par)
        for i in range(len(df_s)):
            fig.add_trace(go.Scatter(
                x=[x_vals.iloc[i], x_vals.iloc[i]],
                y=[y_true[i], y_pred[i]],
                mode="lines",
                line=dict(color=colors[i], width=2),
                hoverinfo="skip",
                showlegend=False
            ))
    
        # pontos REAIS (círculo cheio)
        fig.add_trace(go.Scatter(
            x=x_vals, y=y_true, mode="markers",
            marker=dict(symbol="circle", size=9, line=dict(width=0), color=colors),
            name="Real",
            hovertemplate="%{x}<br>real %{y:.0f}<extra></extra>"
        ))
    
        # pontos PREVISTOS (quadrado vazado)
        fig.add_trace(go.Scatter(
            x=x_vals, y=y_pred, mode="markers",
            marker=dict(symbol="square", size=10, line=dict(width=2, color=colors), color="rgba(0,0,0,0)"),
            name="Previsto",
            hovertemplate="%{x}<br>previsto %{y:.0f}"
                          "<br>|err|/real=%{customdata:.1%}<extra></extra>",
            customdata=ape
        ))
    
        # banda ±10% do real (referência visual)
        fig.add_trace(go.Scatter(
            x=np.concatenate([x_vals, x_vals[::-1]]),
            y=np.concatenate([(1+tol)*y_true, ((1-tol)*y_true)[::-1]]),
            fill="toself", fillcolor="rgba(0,150,0,0.06)",
            line=dict(color="rgba(0,0,0,0)"),
            hoverinfo="skip", showlegend=True, name=f"faixa ±{int(tol*100)}% do real"
        ))
    
        fig.update_layout(
            title=f"Real vs Previsto — {split}  |  Acc@{int(tol*100)}% = {acc10:.1f}%",
            xaxis_title=x_title,
            yaxis_title="toneladas",
            template="plotly_white",
            hovermode="closest"
        )
        fig.show()



    def plot_learning_curve(self):
        loss = self.history_.history.get("loss", [])
        val_loss = self.history_.history.get("val_loss", [])
        epochs = np.arange(1, len(loss)+1)
    
        # NOVO: rótulos conforme a loss escolhida
        lname = "MSE" if self.loss_fn == "mse" else "MAE"
    
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=epochs, y=loss, mode="lines+markers",
            name=f"train {lname}(z)",
            hovertemplate="epoch %{x}<br>"+f"{lname}(z) "+"%{y:.3f}<extra></extra>"
        ))
        if len(val_loss):
            fig.add_trace(go.Scatter(
                x=epochs, y=val_loss, mode="lines+markers",
                name=f"val {lname}(z)",
                hovertemplate="epoch %{x}<br>"+f"val {lname}(z) "+"%{y:.3f}<extra></extra>"
            ))
            best = int(np.argmin(val_loss))
            fig.add_vline(x=epochs[best], line_width=1, line_dash="dash")
            fig.add_annotation(x=epochs[best], y=val_loss[best],
                               text=f"best val @{epochs[best]}: {val_loss[best]:.3f}",
                               showarrow=True, ax=20, ay=-40)
    
        fig.update_layout(
            title=f"Learning Curve — {self.technique.upper()}",
            xaxis_title="epoch",
            yaxis_title=f"{lname} (z-score y)",
            template="plotly_white"
        )
        fig.show()


### Execução dos regressores

In [116]:
reg = AcaiRegressor(
    learning_rate = 1e-2,
    epochs = 300,
    batch_size= 16,
    early_stopping_patience = 30,
    val_frac = 0.3,
    test_frac = 0.2,
    feature_mode="explicit",
    technique="linear", #Por enquanto linear e poly2
    num_cols=["precip_rollsum_8w_end_lag_36w", "week"],
    cat_cols=['periodo'], 
    loss_fn="mae",      
    cyclic_cols={"week":df['mesoregion'].count()},
    include_month_cycle=False,
    include_harmonic_k2=True,
    include_interactions=False,
)
reg.fit(df)
print(reg.feature_names_)


[DEBUG] n_features: 6
[DEBUG] features numéricas: ['precip_rollsum_8w_end_lag_36w', 'week', 'week_sin', 'week_cos']
[DEBUG] categorias OHE: [['entressafra', 'safra']]
Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 2.7586 - mae: 2.7586 - mse: 8.8445 - val_loss: 1.7106 - val_mae: 1.7106 - val_mse: 3.3948
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.6097 - mae: 2.6097 - mse: 7.9741 - val_loss: 1.6021 - val_mae: 1.6021 - val_mse: 2.9994
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.4646 - mae: 2.4646 - mse: 7.1674 - val_loss: 1.4946 - val_mae: 1.4946 - val_mse: 2.6316
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.3198 - mae: 2.3198 - mse: 6.4077 - val_loss: 1.3876 - val_mae: 1.3876 - val_mse: 2.2896
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.1751 - mae: 2.1751 - mse: 5.6938 - val_loss: 1.2807 - val_mae: 1.2807 - val_mse: 1.9729
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.0305 - mae: 2.0305 - mse: 5.0256

In [117]:
metrics = reg.evaluate()
print("Teste: ", metrics["TESTE"])
print("Validação: ", metrics["VALIDACAO"])
print("Treino: ", metrics["TREINO"])

Teste:  {'MSE': 38819376.985905826, 'RMSE': np.float64(6230.519800619032), 'MAE': 4227.307134103126, 'SMAPE': 0.3684900260483123, 'R2': 0.6435732380253636, 'NRMSE_std': np.float64(0.5970148758402658), 'NRMSE_range': np.float64(0.19240580804741586), 'Acc@10%': 0.3157894736842105, 'Acc@20%': 0.5263157894736842, 'Acc@30%': 0.6842105263157895}
Validação:  {'MSE': 7564176.670026835, 'RMSE': np.float64(2750.304832200757), 'MAE': 1693.318953575078, 'SMAPE': 1.3510474096531702, 'R2': 0.7311248899921812, 'NRMSE_std': np.float64(0.5185316865994954), 'NRMSE_range': np.float64(0.12204957035309194), 'Acc@10%': 0.043478260869565216}
Treino:  {'MSE': 14074451.453488141, 'RMSE': np.float64(3751.5931886983885), 'MAE': 2471.1780911006686, 'SMAPE': 0.7230489696633376, 'R2': 0.8421521774924743, 'NRMSE_std': np.float64(0.39730067015736614), 'NRMSE_range': np.float64(0.12391974219695778), 'Acc@10%': 0.23076923076923078}


In [118]:
reg.plot_pairs_real_pred(split="TESTE", x_col="week", tol=0.30)


In [119]:
reg.plot_learning_curve()       

In [26]:
df_future = df.tail(12).copy() 
y_future_hat = reg.predict(df_future)

print(y_future_hat)

[10694.474 12181.418 13720.405 15304.099 16924.967 18575.309 20247.293
 23624.414 26992.389 28652.975 30287.428 33447.06 ]


## Random Forest

In [27]:
from typing import Optional, Literal, Dict, Any, List
from sklearn.ensemble import RandomForestRegressor


class AcaiRandomForest:
    def __init__(
        self,
        # seleção de features
        feature_mode: Literal["engineered", "explicit"] = "explicit",
        num_cols: Optional[List[str]] = None,
        cat_cols: Optional[List[str]] = None,
        cyclic_cols: Optional[Dict[str, int]] = None,  # {"week":52, "month":12}

        # splits
        val_frac: float = 0.2,
        test_frac: float = 0.2,

        # hiperparâmetros do RF
        n_estimators: int = 500,
        max_depth: Optional[int] = None,
        min_samples_split: int = 2,
        min_samples_leaf: int = 1,
        max_features: Optional[Literal["sqrt","log2"]] = "sqrt",
        bootstrap: bool = True,
        oob_score: bool = False,     # pode ligar para oob
        n_jobs: int = -1,
        random_state: int = 42,
    ):
        # seleção de features
        self.feature_mode = feature_mode
        self.num_cols = num_cols or []
        self.cat_cols = cat_cols or []
        self.cyclic_cols = cyclic_cols or {}

        # splits
        self.val_frac = val_frac
        self.test_frac = test_frac

        # RF params
        self.rf_params = dict(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bootstrap,
            oob_score=oob_score,
            n_jobs=n_jobs,
            random_state=random_state,
        )

        # definidos após fit
        self.ohe_: Optional[OneHotEncoder] = None
        self.model_: Optional[RandomForestRegressor] = None
        self.feature_names_: Optional[List[str]] = None
        self.splits_: Dict[str, Any] = {}

        np.random.seed(random_state)

    # ---------- feature engineering ----------
    def _build_numeric_features(self, d: pd.DataFrame) -> np.ndarray:
        names, cols = [], []

        if self.feature_mode == "explicit":
            # (a) numéricas cruas
            for c in self.num_cols:
                if c in d.columns:
                    v = pd.to_numeric(d[c], errors="coerce").to_numpy()
                    cols += [v]; names += [c]
            # (b) expansões cíclicas (sin/cos) para colunas indicadas
            for c, period in self.cyclic_cols.items():
                if c in d.columns:
                    v = pd.to_numeric(d[c], errors="coerce").to_numpy().astype(float)
                    s = np.sin(2*np.pi*v/float(period))
                    c_ = np.cos(2*np.pi*v/float(period))
                    cols += [s, c_]; names += [f"{c}_sin", f"{c}_cos"]
            X_num = np.column_stack(cols) if cols else np.empty((len(d), 0))
            self.feature_names_ = names
            return X_num

        # --- modo "engineered" (compatível com sua outra classe) ---
        week = d["week"].astype(float).to_numpy()
        p = d["precip_rollsum_8w_end_lag_36w"].astype(float).to_numpy()
        den_w = 52.0
        sin_w = np.sin(2*np.pi*week/den_w)
        cos_w = np.cos(2*np.pi*week/den_w)

        cols = [p, sin_w, cos_w]
        names = ["p", "sin_w", "cos_w"]

        if "month" in d.columns:
            month = d["month"].astype(float).to_numpy()
            sin_m = np.sin(2*np.pi*month/12.0)
            cos_m = np.cos(2*np.pi*month/12.0)
            cols += [sin_m, cos_m]; names += ["sin_m", "cos_m"]

        X_num = np.column_stack(cols)
        self.feature_names_ = names
        return X_num

    def _build_X(self, df_train: pd.DataFrame, df_other: pd.DataFrame):
        Xtr_num = self._build_numeric_features(df_train)
        Xot_num = self._build_numeric_features(df_other)

        # decide se usa OHE
        use_ohe = False
        cols_train = None
        if self.feature_mode == "explicit":
            if len(self.cat_cols) > 0:
                use_ohe = True
                cols_train = self.cat_cols
        else:
            # no modo engineered, se existir 'mesoregion', podemos OHE
            if "mesoregion" in df_train.columns:
                use_ohe = True
                cols_train = ["mesoregion"]

        if use_ohe:
            self.ohe_ = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=float)
            Ztr = self.ohe_.fit_transform(df_train[cols_train])
            Zot = self.ohe_.transform(df_other[cols_train])
            Xtr = np.hstack([Xtr_num, Ztr])
            Xot = np.hstack([Xot_num, Zot])
        else:
            Xtr, Xot = Xtr_num, Xot_num

        return Xtr, Xot

    # ---------- split temporal ----------
    def _temporal_split(self, df: pd.DataFrame):
        n = len(df)
        cut_test = int(n * (1 - self.test_frac))
        cut_val  = int(cut_test * (1 - self.val_frac))
        df_train = df.iloc[:cut_val].copy()
        df_val   = df.iloc[cut_val:cut_test].copy()
        df_test  = df.iloc[cut_test:].copy()
    
        y_train = pd.to_numeric(df_train[self.target_col], errors="coerce").to_numpy(float)
        y_val   = pd.to_numeric(df_val[self.target_col],   errors="coerce").to_numpy(float)
        y_test  = pd.to_numeric(df_test[self.target_col],  errors="coerce").to_numpy(float)
    
        X_train, X_val = self._build_X(df_train, df_val)

        # TESTE: mesma decisão de OHE do _build_X
        X_test_num = self._build_numeric_features(df_test)
        use_ohe = (self.ohe_ is not None)
        if use_ohe:
            # colunas categóricas usadas no fit
            cols_test = self.cat_cols if self.feature_mode == "explicit" else ["mesoregion"]
            cols_test = [c for c in cols_test if c in df_test.columns]
            Ztest = self.ohe_.transform(df_test[cols_test])
            X_test = np.hstack([X_test_num, Ztest])
        else:
            X_test = X_test_num

        self.splits_ = dict(
            df_train=df_train, df_val=df_val, df_test=df_test,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )

    # ---------- API ----------
    def fit(self, df: pd.DataFrame):
        """
        df precisa conter: 'week_start', 'toneladas_semana' e as colunas listadas
        em num_cols, cat_cols e cyclic_cols (quando feature_mode='explicit').
        """
        # colunas necessárias mínimas
        req = set(["toneladas_semana", "week_start"])
        if self.feature_mode == "explicit":
            req |= set(self.num_cols) | set(self.cat_cols) | set(self.cyclic_cols.keys())

        missing = [c for c in req if c not in df.columns]
        if missing:
            raise ValueError(f"Colunas ausentes: {missing}")

        # filtra para as colunas usadas e ordena temporalmente
        use_cols = sorted(list(req))
        df2 = df.loc[:, use_cols].dropna(subset=use_cols).sort_values("week_start").copy()

        self._temporal_split(df2)

        # treina o RF
        self.model_ = RandomForestRegressor(**self.rf_params)
        self.model_.fit(self.splits_["X_train"], self.splits_["y_train"])
        return self

    def predict(self, df_new: pd.DataFrame) -> np.ndarray:
        X_num = self._build_numeric_features(df_new)
        if self.ohe_ is not None:
            cols_infer = self.cat_cols if self.feature_mode == "explicit" else ["mesoregion"]
            cols_infer = [c for c in cols_infer if c in df_new.columns]
            Z = self.ohe_.transform(df_new[cols_infer])
            X = np.hstack([X_num, Z])
        else:
            X = X_num
        return self.model_.predict(X).astype(float)

    def _metrics(self, y_true, y_hat) -> Dict[str, float]:
        mse  = mean_squared_error(y_true, y_hat)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(y_true, y_hat)
        mape = mean_absolute_percentage_error(y_true, y_hat)
        r2   = r2_score(y_true, y_hat)
        nrmse_std = rmse / (np.std(y_true) + 1e-9)
        nrmse_rng = rmse / (np.max(y_true) - np.min(y_true) + 1e-9)
        return dict(MSE=mse, RMSE=rmse, MAE=mae, MAPE=mape, R2=r2,
                    NRMSE_std=nrmse_std, NRMSE_range=nrmse_rng)

    def _split_arrays_(self, split="TESTE"):
        s = self.splits_
        X = s["X_test"] if split=="TESTE" else s["X_val"] if split=="VALIDACAO" else s["X_train"]
        y = s["y_test"] if split=="TESTE" else s["y_val"] if split=="VALIDACAO" else s["y_train"]
        yhat = self.model_.predict(X).astype(float)
        return y, yhat
    
    def accuracy_table(self, split="TESTE", tol_list=(0.05, 0.10, 0.20)):
        y, yhat = self._split_arrays_(split)
        mae  = mean_absolute_error(y, yhat)
        mape = mean_absolute_percentage_error(y, yhat)
        def acc(tol):
            rel = np.abs(yhat - y) / (np.abs(y) + 1e-9)
            return float((rel <= tol).mean())
        return {
            "split": split,
            "MAE": mae,
            "MAPE": mape,
            **{f"Acc@{int(t*100)}%": acc(t) for t in tol_list}
        }
    
    def plot_parity(self, split="TESTE", tol=0.10, title=None):
        import plotly.graph_objects as go
        y, yhat = self._split_arrays_(split)
        rel = np.abs(yhat - y) / (np.abs(y) + 1e-9)
        inside = (rel <= tol)
    
        acc = float(inside.mean())
        title = title or f"Parity plot — {split} (Acc@{int(tol*100)}% = {acc*100:.1f}%)"
    
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=y, y=yhat, mode="markers",
            marker=dict(size=8, opacity=0.8),
            name="pontos",
            marker_color=np.where(inside, "green", "red"),
            hovertemplate="real %{x:.0f}<br>prev %{y:.0f}<br>|err|/real=%{customdata:.1%}<extra></extra>",
            customdata=rel
        ))
        # linha y=x
        lo, hi = float(np.min(y)), float(np.max(y))
        fig.add_trace(go.Scatter(x=[lo, hi], y=[lo, hi], mode="lines", name="y=x", line=dict(dash="dash")))
        fig.update_layout(
            title=title,
            xaxis_title="Real (toneladas)",
            yaxis_title="Previsto (toneladas)",
            legend_title="",
            template="plotly_white"
        )
        fig.show()
    
    def plot_accuracy_curve(self, split="TESTE", max_tol=0.50):
        import plotly.graph_objects as go
        y, yhat = self._split_arrays_(split)
        rel = np.abs(yhat - y) / (np.abs(y) + 1e-9)
    
        tols = np.linspace(0, max_tol, 51)
        accs = [(rel <= t).mean() for t in tols]
    
        fig = go.Figure(go.Scatter(x=tols*100, y=np.array(accs)*100, mode="lines+markers"))
        fig.update_layout(
            title=f"Accuracy vs. tolerância — {split}",
            xaxis_title="Tolerância (%)",
            yaxis_title="Accuracy (%)",
            template="plotly_white"
        )
        fig.add_vline(x=10, line_dash="dot")
        fig.add_annotation(x=10, y=np.interp(0.10, tols, accs)*100,
                           text=f"Acc@10% = {np.interp(0.10, tols, accs)*100:.1f}%",
                           showarrow=True, ax=30, ay=-30)
        fig.show()


    def evaluate(self) -> Dict[str, Dict[str, float]]:
        Xtr = self.splits_["X_train"]; ytr = self.splits_["y_train"]
        Xva = self.splits_["X_val"];   yva = self.splits_["y_val"]
        Xte = self.splits_["X_test"];  yte = self.splits_["y_test"]

        ytr_hat = self.model_.predict(Xtr)
        yva_hat = self.model_.predict(Xva)
        yte_hat = self.model_.predict(Xte)

        def acc_within(y_true, y_hat, tol=0.10):
            rel = np.abs(y_hat - y_true) / (np.abs(y_true) + 1e-9)
            return float((rel <= tol).mean())

        return dict(
            TREINO   ={**self._metrics(ytr, ytr_hat), "Acc@10%": acc_within(ytr, ytr_hat)},
            VALIDACAO={**self._metrics(yva, yva_hat), "Acc@10%": acc_within(yva, yva_hat)},
            TESTE    ={**self._metrics(yte, yte_hat), "Acc@10%": acc_within(yte, yte_hat)},
        )

    # importância de features
    def plot_feature_importance(self, top_k: int = 15):
        if not hasattr(self.model_, "feature_importances_"):
            print("Este modelo não possui feature_importances_.")
            return
        fi = self.model_.feature_importances_.ravel()
        # nomes: numéricas + (eventuais) dummies
        names = list(self.feature_names_ or [])
        if self.ohe_ is not None:
            # concatena categorias de todas as colunas categóricas
            cat_names = []
            for cats in self.ohe_.categories_:
                cat_names += list(map(str, cats))
            names += cat_names

        order = np.argsort(fi)[::-1][:top_k]
        fig = go.Figure(go.Bar(x=fi[order][::-1], y=[names[i] for i in order][::-1], orientation="h"))
        fig.update_layout(title="Importância de Features (Random Forest)",
                          xaxis_title="Gini importance", yaxis_title="")
        fig.show()


In [28]:
rf = AcaiRandomForest(
    feature_mode="explicit",
    num_cols=["precip_rollsum_8w_end_lag_36w"],
    cat_cols=[],
    cyclic_cols={"week": 52},
    n_estimators=600, max_depth=None, min_samples_leaf=2,
    max_features="sqrt", oob_score=True, random_state=42
)

rf.fit(df)


print(rf.accuracy_table(split="TREINO"))

# 2) Parity plot com destaque ±10%
rf.plot_parity(split="TESTE", tol=0.10)

# 3) Curva de accuracy vs tolerância (0–50%)
rf.plot_accuracy_curve(split="TESTE", max_tol=0.50)

# (opcional) importância de features
rf.plot_feature_importance(top_k=15)


AttributeError: 'AcaiRandomForest' object has no attribute 'target_col'